# 12장 인공지능 PDF 문서 요약기

## 12.1 PDF 문서 읽기

[12장: 317페이지]

In [4]:
from PyPDF2 import PdfReader

pdf_file = 'C:/myPyAI/data/2102.12092v1.pdf' # PDF 파일 경로
reader = PdfReader(pdf_file) # PDF 문서 읽기
meta = reader.metadata   # PDF 문서의 메타데이터 가져옴(문서 제목, 저자 등)
page = reader.pages[0]   # 첫 페이지 내용 가져옴 
page_text = page.extract_text() # 페이지의 텍스트 추출 

print("- 문서 제목:", meta.title)
print("- 문서 저자:", meta.author)
print("- 첫 페이지 내용 일부:\n",  page_text[:300])

- 문서 제목: Zero-Shot Text-to-Image Generation
- 문서 저자: Aditya Ramesh, Mikhail Pavlov, Gabriel Goh, Scott Gray, Chelsea Voss, Alec Radford, Mark Chen, Ilya Sutskever
- 첫 페이지 내용 일부:
 Zero-Shot Text-to-Image Generation
Aditya Ramesh1Mikhail Pavlov1Gabriel Goh1Scott Gray1
Chelsea Voss1Alec Radford1Mark Chen1Ilya Sutskever1
Abstract
Text-to-image generation has traditionally fo-
cused on ﬁnding better modeling assumptions for
training on a ﬁxed dataset. These assumptions
might invo


## 12.2 PDF 문서 요약하기

[12장: 318페이지]

In [5]:
from PyPDF2 import PdfReader
import tiktoken

pdf_file = 'C:/myPyAI/data/President_Obamas_Farewell_Address_영어_원본.pdf' # PDF 파일 경로
reader = PdfReader(pdf_file) # PDF 문서 읽기

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

page_token_nums = []
for page in reader.pages:
    page_text = page.extract_text() # 페이지의 텍스트 추출
    token_num = len(enc.encode(page_text)) # 페이지마다 토큰 개수 구하기
    page_token_nums.append(token_num)

print("- 각 페이지의 토큰 수:", page_token_nums)
print("- 전체 페이지에서 최대 토큰 수:", max(page_token_nums))
print("- 전체 페이지의 토큰 수 합계:", sum(page_token_nums))

- 각 페이지의 토큰 수: [694, 699, 734, 754, 719, 694, 698, 745, 809, 256]
- 전체 페이지에서 최대 토큰 수: 809
- 전체 페이지의 토큰 수 합계: 6802


[12장: 319페이지]

In [6]:
import openai
import os

# OpenAI 라이브러리를 이용해 텍스트를 요약하는 함수
def summarize_text(user_text, lang="en"): # lang 인자에 영어를 기본적으로 지정
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    if lang == "en":
        messages = [ 
            {"role": "system", "content": "You are a helpful assistant in the summary."},
            {"role": "user", "content": f"Summarize the following. \n {user_text}"}
        ]
    elif lang == "ko":
        messages = [ 
            {"role": "system", "content": "You are a helpful assistant in the summary."},
            {"role": "user", "content": f"다음의 내용을 한국어로 요약해 주세요 \n {user_text}"}
#             {"role": "user", "content": f"Summarize the following in Korea. \n {user_text}"}
        ]
        
    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=2000,  # 응답 최대 토큰 수 지정 
                            temperature=0.3,  # 완성의 다양성을 조절하는 온도 설정
                            n=1              # 생성할 완성의 개수 지정
    )     
    summary = response["choices"][0]["message"]["content"]
    return summary

[12장: 320페이지]

In [7]:
from PyPDF2 import PdfReader
import textwrap

pdf_file = 'C:/myPyAI/data/President_Obamas_Farewell_Address_영어_원본.pdf' # PDF 파일 경로
reader = PdfReader(pdf_file) # PDF 문서 읽기
page = reader.pages[0]      # 첫 페이지 내용 가져옴 
page_text = page.extract_text() # 페이지의 텍스트 추출 

summary = summarize_text(page_text) # 첫 페이지 요약 (영어)

# 텍스트 축약
shorten_summary = textwrap.shorten(summary, 250, placeholder=' [..이하 생략..]')

print("[페이지 요약(축약)]\n", shorten_summary) # 요약 내용 출력(축약)
# print("[페이지 요약]\n", summary) # 요약 내용 출력

[페이지 요약(축약)]
 President Obama delivered his Farewell Address in Chicago on January 10, 2017, expressing gratitude to the American people for their support and engagement during his presidency. He reflected on his early days in Chicago, emphasizing the [..이하 생략..]


In [8]:
summary = summarize_text(page_text, "ko") # 첫 페이지 요약 (한국어)

# 텍스트 축약
shorten_summary = textwrap.shorten(summary, 250, placeholder=' [..이하 생략..]')

print("[페이지 요약(축약)]\n", shorten_summary) # 요약 내용 출력(축약)
print("[페이지 요약]\n", summary) # 요약 내용 출력

[페이지 요약(축약)]
 2017년 1월 10일 대통령 오바마의 작별 연설 내용은 다음과 같습니다. 오바마 대통령은 시카고에서 연설을 하며 미국 국민들에게 감사의 말을 전했습니다. 그동안 국민들과의 대화를 통해 더 나은 대통령이 되었고, 더 나은 사람이 되었다고 말했습니다. 그는 시카고에서 자신의 삶의 목적을 찾고 있는 20대 초반 때 교회 단체와 함께 일하며 미국인들의 힘과 존엄성을 목격했다고 이야기했습니다. 또한, 미국의 자유와 민주주의에 대한 [..이하 생략..]
[페이지 요약]
 2017년 1월 10일 대통령 오바마의 작별 연설 내용은 다음과 같습니다. 오바마 대통령은 시카고에서 연설을 하며 미국 국민들에게 감사의 말을 전했습니다. 그동안 국민들과의 대화를 통해 더 나은 대통령이 되었고, 더 나은 사람이 되었다고 말했습니다. 그는 시카고에서 자신의 삶의 목적을 찾고 있는 20대 초반 때 교회 단체와 함께 일하며 미국인들의 힘과 존엄성을 목격했다고 이야기했습니다. 또한, 미국의 자유와 민주주의에 대한 중요성을 강조하며 시민의 의무와 목적에 대해 이야기했습니다.


[12장: 321페이지]

In [9]:
import openai
from PyPDF2 import PdfReader
import tiktoken
import os

pdf_file = 'C:/myPyAI/data/President_Obamas_Farewell_Address_영어_원본.pdf' # PDF 파일 경로

reader = PdfReader(pdf_file) # PDF 문서 읽기

text_summaries = []

for page in reader.pages:
    page_text = page.extract_text() # 페이지의 텍스트 추출
    text_summary = summarize_text(page_text)
    text_summaries.append(text_summary)

print("- 요약 개수:", len(text_summaries))
print("- 페이지별 요약문:", text_summaries)

- 요약 개수: 10
- 페이지별 요약문: ["President Obama delivered his Farewell Address in Chicago on January 10, 2017. He expressed gratitude to the American people for their support and engagement during his presidency. Reflecting on his early days in Chicago, he highlighted the importance of community involvement and the power of ordinary people coming together to bring about change. Obama emphasized the core American values of equality, liberty, and the pursuit of happiness, stressing the importance of citizen engagement in shaping a more perfect union. He acknowledged the nation's history of striving for a common good and encouraged continued active citizenship for future generations.", "The speaker reflects on America's exceptionalism, highlighting the nation's capacity for change and progress. They acknowledge the challenges and setbacks faced in the journey towards a more inclusive society. The accomplishments achieved during the speaker's tenure, such as economic recovery, diplomatic achieve

[12장: 322페이지]

In [10]:
import textwrap
import tiktoken

# 요약 리스트를 최종적으로 요약하는 함수
def summarize_text_final(text_list, lang = 'en'):
    # 리스트를 연결해 하나의 요약 문자열로 통합
    joined_summary = " ".join(text_list) 

    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_num = len(enc.encode(joined_summary)) # 텍스트 문자열의 토큰 개수 구하기

    req_max_token = 2000 # 응답을 고려해 설정한 최대 요청 토큰    
    final_summary = "" # 빈 문자열로 초기화
    if token_num < req_max_token: # 설정한 토큰보다 작을 때만 실행 가능
        # 하나로 통합한 요약문을 다시 요약
        final_summary = summarize_text(joined_summary, lang)
        
    return token_num, final_summary

In [14]:
lang = 'en' # 영어로 요약
token_num, final_summary = summarize_text_final(text_summaries, lang)

if final_summary != "":
    shorten_final_summary = textwrap.shorten(final_summary, 
                                             250, 
                                             placeholder=' [..이하 생략..]')
    print("- 통합한 페이지 요약의 토큰 수:", token_num)
    print("- 최종 요약(축약)\n", shorten_final_summary) # 최종 요약문 출력(축약)
    print(final_summary )

- 통합한 페이지 요약의 토큰 수: 1088
- 최종 요약(축약)
 President Obama delivered his Farewell Address in Chicago, expressing gratitude to the American people for their support during his presidency. He highlighted the importance of community involvement and citizen engagement in bringing [..이하 생략..]
President Obama delivered his Farewell Address in Chicago, expressing gratitude to the American people for their support during his presidency. He highlighted the importance of community involvement and citizen engagement in bringing about change, emphasizing core American values like equality and liberty. The speech celebrated accomplishments in economic recovery, diplomacy, and social progress, while acknowledging challenges such as economic inequality and racial divisions. Obama called for unity, democracy, and solidarity in facing future challenges, stressing the need for a new social compact to address changing economic landscapes and support the middle class. He urged for empathy, understanding, and i

[12장: 323페이지]

In [15]:
lang = 'ko' # 한국어로 요약
token_num, final_summary_ko = summarize_text_final(text_summaries, lang)

if final_summary_ko != "":
    shorten_final_summary_ko = textwrap.shorten(final_summary_ko, 
                                                250, 
                                                placeholder=' [..이하 생략..]')
    print("- 통합한 페이지 요약의 토큰 수:", token_num)
    print("- 최종 요약(축약)\n", shorten_final_summary_ko) # 최종 요약문 출력(축약)

- 통합한 페이지 요약의 토큰 수: 1088
- 최종 요약(축약)
 2017년 1월 10일, 오바마 대통령은 시카고에서 이별 연설을 진행했습니다. 그는 대통령으로 임기를 보낼 때 미국 국민들에게 지지와 참여에 대한 감사를 표현했습니다. 시카고 초기의 경험을 회상하며 지역사회 참여의 중요성과 평범한 사람들이 변화를 이끌어내는 힘을 강조했습니다. 오바마는 평등, 자유, 행복의 미국 핵심 가치를 강조하며 시민 참여의 중요성을 강조하여 더 완벽한 연합을 형성하는 데 기여하는 것을 강조했습니다. [..이하 생략..]


## 12.3 요약한 내용 번역하기

[12장: 324페이지]

In [16]:
import openai
import os

# OpenAI 라이브러리를 이용해 영어를 한국어로 번역하는 함수
def traslate_english_to_korean_using_openAI(text):    
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Translate the following English sentences into Korean.\n {text}"
    messages = [ {"role": "user", "content": user_content} ]
    
    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages,     # 전달할 메시지 지정
                            max_tokens=2000,       # 응답 최대 토큰 수 지정 
                            temperature=0.3,       # 완성의 다양성을 조절하는 온도 설정
                            n=1                    # 생성할 완성의 개수 지정
    )

    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기
    
    return assistant_reply

In [17]:
e_text = final_summary
k_text = traslate_english_to_korean_using_openAI(e_text) # 영어를 한국어로 번역

# 텍스트 축약
shorten_k_text = textwrap.shorten(k_text, 200, placeholder=' [..이하 생략..]')

print("[요약 내용 한글 번역(OpenAI 이용)]\n", shorten_k_text) # 번역 내용 출력(축약)

[요약 내용 한글 번역(OpenAI 이용)]
 오바마 대통령은 시카고에서 이별 연설을 했는데, 미국 국민들에게 대통령으로 재임하는 동안의 지원에 감사를 표했습니다. 그는 변화를 이끌기 위해 공동체 참여와 시민 참여의 중요성을 강조하며, 평등과 자유와 같은 미국의 핵심 가치를 강조했습니다. 연설은 경제 회복, 외교, 사회적 발전에서의 성취를 축하하면서, 경제적 불평등과 인종 갈등과 [..이하 생략..]


[12장: 325페이지]

In [19]:
import deepl # deepl 라이브러리를 임포트
import os

# DeepL 라이브러리를 이용해 텍스트를 한국어로 번역하는 함수
def traslate_english_to_korean_using_deepL(text):   
    auth_key = os.environ["DEEPL_AUTH_KEY"] # DeepL 인증 키
    translator = deepl.Translator(auth_key) # translator 객체를 생성

    # 번역 결과 객체를 result 변수에 할당
    result = translator.translate_text(text, target_lang="KO") 
    
    return result.text

In [21]:
e_text = final_summary
k_text = traslate_english_to_korean_using_deepL(e_text) # 영어를 한국어로 번역

# 텍스트 축약
shorten_k_text = textwrap.shorten(k_text, 200, placeholder=' [..이하 생략..]')

print("[요약 내용 한글 번역(DeepL 이용)]\n", shorten_k_text) # 번역 내용 출력(축약)

[요약 내용 한글 번역(DeepL 이용)]
 오바마 대통령은 시카고에서 고별 연설을 통해 재임 기간 동안 미국 국민들의 지지에 감사를 표했습니다. 그는 평등과 자유와 같은 미국의 핵심 가치를 강조하면서 변화를 가져오는 데 있어 지역사회 참여와 시민 참여의 중요성을 강조했습니다. 이 연설에서는 경제 회복, 외교, 사회 발전의 성과를 축하하는 한편, 경제적 불평등과 인종 분열과 같은 [..이하 생략..]


[12장: 326페이지]

In [22]:
%%writefile C:\myPyAI\code\my_text_sum.py
# 텍스트 요약을 위한 모듈

import openai
import os
import deepl
import tiktoken

# OpenAI 라이브러리를 이용해 텍스트를 요약하는 함수
def summarize_text(user_text, lang="en"): # lang 인자에 영어를 기본적으로 지정
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    if lang == "en":
        messages = [ 
            {"role": "system", "content": "You are a helpful assistant in the summary."},
            {"role": "user", "content": f"Summarize the following. \n {user_text}"}
        ]
    elif lang == "ko":
        messages = [ 
            {"role": "system", "content": "You are a helpful assistant in the summary."},
            {"role": "user", "content": f"다음의 내용을 한국어로 요약해 주세요 \n {user_text}"}
#             {"role": "user", "content": f"Summarize the following in Korea. \n {user_text}"}
        ]
        
    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=2000,  # 응답 최대 토큰 수 지정 
                            temperature=0.3,  # 완성의 다양성을 조절하는 온도 설정
                            n=1              # 생성할 완성의 개수 지정
    )     
    summary = response["choices"][0]["message"]["content"]
    return summary

# 요약 리스트를 최종적으로 요약하는 함수
def summarize_text_final(text_list, lang = 'en'):
    # 리스트를 연결해 하나의 요약 문자열로 통합
    joined_summary = " ".join(text_list) 

    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
    token_num = len(enc.encode(joined_summary)) # 텍스트 문자열의 토큰 개수 구하기

    req_max_token = 2000 # 응답을 고려해 설정한 최대 요청 토큰    
    final_summary = "" # 빈 문자열로 초기화
    if token_num < req_max_token: # 설정한 토큰보다 작을 때만 실행 가능
        # 하나로 통합한 요약문을 다시 요약
        final_summary = summarize_text(joined_summary, lang)
        
    return token_num, final_summary

# OpenAI 라이브러리를 이용해 영어를 한국어로 번역하는 함수
def traslate_english_to_korean_using_openAI(text):    
    # API 키 설정
    openai.api_key = os.environ["OPENAI_API_KEY"]

    # 대화 메시지 정의
    user_content = f"Translate the following English sentences into Korean.\n {text}"
    messages = [ {"role": "user", "content": user_content} ]
    
    # Chat Completions API 호출
    response = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo", # 사용할 모델 선택 
                            messages=messages, # 전달할 메시지 지정
                            max_tokens=2000,  # 응답 최대 토큰 수 지정 
                            temperature=0.3,  # 완성의 다양성을 조절하는 온도 설정
                            n=1               # 생성할 완성의 개수 지정
    )

    assistant_reply = response.choices[0].message['content'] # 첫 번째 응답 결과 가져오기
    
    return assistant_reply

# DeepL 라이브러리를 이용해 텍스트를 한국어로 번역하는 함수
def traslate_english_to_korean_using_deepL(text):   
    auth_key = os.environ["DEEPL_AUTH_KEY"] # Deepl 인증 키
    translator = deepl.Translator(auth_key) # translator 객체를 생성

    result = translator.translate_text(text, target_lang="KO") # 번역 결과 객체를 result 변수에 할당
    
    return result.text

Writing C:\myPyAI\code\my_text_sum.py


## 12.4 PDF 문서를 요약하는 웹 앱

[12장: 328페이지]

In [23]:
%%writefile C:\myPyAI\code\st\pdf_summary_app.py
# PDF 문서를 요약하는 웹 앱

import my_text_sum # 텍스트를 요약하기 위한 모듈
import streamlit as st
import openai
import os
from PyPDF2 import PdfReader
import tiktoken
import textwrap

# PDF 파일을 요약하는 함수
def summarize_PDF_file(pdf_file, lang, trans_checked):
    if (pdf_file is not None):
        st.write("PDF 문서를 요약 중입니다. 잠시만 기다려 주세요.") 
        reader = PdfReader(pdf_file) # PDF 문서 읽기

        text_summaries = []
        
        for page in reader.pages:
            page_text = page.extract_text() # 페이지의 텍스트 추출
            text_summary = my_text_sum.summarize_text(page_text, lang)
            text_summaries.append(text_summary)
            
        token_num, final_summary = my_text_sum.summarize_text_final(text_summaries, lang)
        
        if final_summary != "":
            shorten_final_summary = textwrap.shorten(final_summary, 
                                                     250, 
                                                     placeholder=' [..이하 생략..]')

            st.write("- 최종 요약(축약):", shorten_final_summary) # 최종 요약문 출력 (축약)
            #st.write("- 최종 요약:", shorten_final_summary) # 최종 요약문 출력

            if trans_checked:
                trans_result = my_text_sum.traslate_english_to_korean_using_openAI(final_summary)
                shorten_trans_result = textwrap.shorten(trans_result, 
                                                        200, 
                                                        placeholder=' [..이하 생략..]')
                st.write("- 한국어 요약(축약):", shorten_trans_result) # 한국어 번역문 출력 (축약)
                #st.write("- 한국어 요약:", trans_result) # 한국어 번역문 출력
        else:
            st.write("- 통합한 요약문의 토큰 수가 커서 요약할 수 없습니다.")

# ------------- 메인 화면 구성 --------------------------  
st.title("PDF 문서를 요약하는 웹 앱")

uploaded_file = st.file_uploader("PDF 파일을 업로드하세요.", type='pdf')

radio_selected_lang = st.radio('PDF 문서 언어', ['한국어', '영어'], index=1, horizontal=True)

if radio_selected_lang == '영어':
    lang_code = 'en' 
    checked = st.checkbox('한국어 번역 추가') # 체크박스 생성
else:
    lang_code = 'ko' 
    checked = False # 체크박스 불필요
    
clicked = st.button('PDF 문서 요약')

if clicked:
    summarize_PDF_file(uploaded_file, lang_code, checked) # PDF 파일 요약 수행

Writing C:\myPyAI\code\st\pdf_summary_app.py


## 12.5 정리